In [1]:
from ml_training import settings
from ml_training.models.encoders.tf_idf_encoder import A
from ml_training.models.encoders.hub_averaging_encoder import HubEncoder
from ml_training.models.encoders.tf_idf_encoder import TextEncoder
from ml_training.models.predictors.ranking_nn import RankingModel


# bow_dssm = BoWDSSM(
#     TextEncoder.load(settings.Settings.models_dir / 'text_encoder.pt'),
#     HubEncoder.load(settings.Settings.models_dir / 'hub_encoder.pt'),
#     RankingModel.load(settings.Settings.models_dir / 'bow_dssm_1.pt')
# )

ImportError: cannot import name 'A' from 'ml_training.models.encoders.tf_idf_encoder' (/home/sergiy/.cache/pypoetry/virtualenvs/habr-article-analyzer-f8Ei3dB1-py3.11/lib/python3.11/site-packages/ml_training/models/encoders/tf_idf_encoder.py)